In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase
from pandas import read_csv 

dataset_fpath = "datasets/dbpedia.tsv"

df = read_csv(dataset_fpath, sep="\t", encoding="utf-8")
bl = BaselineLinker()

for i, row in df.iterrows():
    phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                       for phrase in row.targets.split(",")]
        
    print("\n\n{}\n".format(row.context))
    
    for phrase, candidate in bl.link(row.context, phrases):
        link = candidate.link if candidate else ""
        print(phrase.text, link)

In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase

phrases = "New York, windows, catwalk, teardrops, commuters, curved, floor, shape, video, walls, Grand Central Terminal"
context = "Inside, it’s even wackier: curved walls, windows in the shape of teardrops, and a catwalk with a tiny video screen embedded in the floor that shows an endless loop of antlike commuters rushing through Grand Central Terminal in New York."

phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
bl = BaselineLinker()
for phrase, candidate in bl.link(context, phrases):
    print(phrase.text, candidate.link)

In [1]:
from linkers.baseline import BaselineLinker
from candidate import Phrase

context = "San Francisco said the visit would serve as a cornerstone for future interaction between players and coaches from the Nets and young Russians, with the aim of developing basketball in Russia, where the sport is a distant third in popularity behind soccer and hockey."
phrases = "San Francisco"

phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
bl = BaselineLinker()

for phrase, candidate in bl.link(context, phrases):
    print(phrase.text, candidate)

>>> 0.6878816097554294=log(7650.0)*0.07692307692307693 Candidate(score=0.6878816097554294, name='Albuquerque', link='http://dbpedia.org/resource/Albuquerque,_New_Mexico', wiki='', types=['Place', 'AdministrativeArea'], names=['अल्बुकर्क', 'Albuquerque', 'Albuquerque, Novi Meksiko', 'ალბუქერქე', 'Ալբուկերկե', 'Albuquerque, New Mexico', 'Albuquerque, Nieu-Meksiko', 'አልበከርኪ', '阿布奎基', 'ألباكركي', 'アルバカーキ', 'Albukerke', 'Albuquerque (kapital sa kondado sa Estados Unidos)', 'Альбукерке', 'Горад Альбукерке', 'Албакърки', 'البوکرکی', 'আলবাকার্কি', 'ალბუკერკე', 'ඇල්බර්කර්කි', 'Albuquerque, Nova Mehika', 'Албукерки', 'Албюкуэрокю', 'ಆಲ್ಬುಕರ್ಕ್ (ನ್ಯೂ ಮೆಕ್ಸಿಕೋ)', '앨버커키', 'Albuquerque, Meksiko Nowydh', 'ஆல்புகெர்க்கி', 'అల్బుకెర్కీ', 'แอลบูเคอร์คี', 'Albuquerque, Mecsico Newydd', 'Албукерке', 'Albukerkė', 'Albukwér kwé', 'آلبوکرک، نیو میکسیکو', 'Албакерки', 'आल्बुकर्की', 'Αλμπουκέρκι', 'آلبوکرکی', 'Beeʼeldííl Dahsinil', 'Albuquerque, Novo México', 'આલ્બુકર્ક્યુ', 'ਅਲਬੂਕਰਕੀ', 'אלבקרקי', 'Beeldil Dae

>>> 0.5625554143915616=log(1500.0)*0.07692307692307693 Candidate(score=0.5625554143915616, name='The Castro', link='http://dbpedia.org/resource/Castro_District,_San_Francisco', wiki='', types=['Place', 'AdministrativeArea'], names=['The Castro', 'Кастро', 'The Castro, San Francisco', 'Castro', 'Kastro rajons', 'Le Castro', '캐스트로', '卡斯楚街', 'Ardal Castro', 'カストロ通り', 'הקסטרו', 'Castro District', 'Castro Street', 'Castro District, The Castro', 'The Castro, California'], uris={'wikidata.org/entity/Q956169'})
>>> 0.38312204396919053=log(1450.0)*0.05263157894736842 Candidate(score=0.38312204396919053, name='Tomave Municipality', link='http://dbpedia.org/resource/Tomave_Municipality', wiki='', types=['Place', 'AdministrativeArea'], names=['ტომავეშ მუნიციპალიტეტი', 'Municipio Tomave', 'Tumawi munisipyu', 'ტომავეს მუნიციპალიტეტი', 'Tomave Municipality', 'Tomave', 'San Francisco Tarana kantun', 'Tolapampa', 'Tumawi', 'Cuchagua kantun', 'Viluyo kantun', 'Tomave munisipyu', 'Tacora kantun', 'Ubina'

In [ ]:
%load_ext autoreload
%autoreload 2

# DQL

# Testing type of links